In [27]:
#import statements 
import glob
import random
from typing import List
from collections import defaultdict

import numpy as np
from numpy.random import choice

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from symusic import Score
from miditok import REMI, TokenizerConfig
from midi2audio import FluidSynth # Import library
from IPython.display import Audio, display

Task 1 <br>
This assignment focuses on symbolic music modeling. The goal is to train a model that learns a distribution \( p(x) \) over symbolic music data (e.g., MIDI)  specifically within the EDM genre. In addition it is capable of sampling new sequences from this learned distribution unconditionally. We will be using the LSTM model for this task. <br>

Get list of files for training/test sets

In [28]:
import os
import glob
print("CWD:", os.getcwd())
print("Train directory exists?", os.path.exists("./train"))
print("Train files (glob):", glob.glob("./train/*.midi"))

train_files = glob.glob("./train/*.midi")
test_files = glob.glob("./test/*.midi")

CWD: c:\Users\sammy\Downloads\cse 153_task1\cse153_task1
Train directory exists? True
Train files (glob): ['./train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav.midi', './train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_02_Track02_wav.midi', './train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_03_Track03_wav.midi', './train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_04_Track04_wav.midi', './train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_05_Track05_wav.midi', './train\\MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--1.midi', './train\\MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--2.midi', './train\\MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--3.midi', './train\\MIDI-Unprocessed_01_R1_2009_01-04_ORIG_MID--AUDIO_01_R1_2009_01_R1_2009_01_WAV.midi', './train\\MIDI-Unprocessed_01_R1_2009_01-04_ORIG_MID--AUDIO_01_R1_2009_01_R1_

Construct a PyTorch Dataset

In [29]:
class MIDIDataset(Dataset):
    def __init__(self, file_paths: List[str], tokenizer):
        self.tokenizer = tokenizer
        self.file_paths = file_paths
        
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        midi = Score(self.file_paths[idx])
        MAX_LEN = 512  # or even 512 to be safe
        tokens = self.tokenizer(midi)
        tokens = tokens[:MAX_LEN]
        # Return as dictionary to match training function expectations
        return {'input_ids': torch.tensor(tokens, dtype=torch.long)}

Configure the Tokenizer in order to be use to 

In [ ]:
config = TokenizerConfig(
    num_velocities=32,           # Classical dynamics (was 1)
    use_chords=True,            # Essential for harmony (was False)
    use_programs=False,         # Piano only (was True)
    use_time_signatures=True,   # Classical changes time sigs
    use_rests=True,             # Important in classical
)
tokenizer = REMI(config)
tokenizer.train(vocab_size=1500, files_paths=train_files)
tokenizer.save("tokenizer.json")

Define PyTorch datasets and dataloaders

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from miditok.pytorch_data import DatasetMIDI, DataCollator
dataset = MIDIDataset(train_files, tokenizer)


train_dataset = DatasetMIDI(
    files_paths=train_files,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
test_dataset = DatasetMIDI(
    files_paths=test_files,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
print(f"# Train files loaded: {len(train_dataset)}")
print(f"# Test files loaded: {len(test_dataset)}")

collator = DataCollator(tokenizer.pad_token_id)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collator)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=collator)


# Train files loaded: 938
# Test files loaded: 105


In [ ]:
#for rhythm
def extract_position_ids(token_sequence, tokenizer):
    position_ids = []
    current_pos = 0  # default if no position is found yet

    for token in token_sequence:
        token_str = tokenizer.vocab[token]
        if token_str.startswith("Position_"):
            try:
                current_pos = int(token_str.split("_")[1])
            except:
                current_pos = 0
        position_ids.append(current_pos)

    return torch.tensor(position_ids, dtype=torch.long)



LSTM Model<br>

In [ ]:
class MusicRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=512, num_layers=2,
                 dropout=0.3, bidirectional=False, max_position_embeddings=64):
        super(MusicRNN, self).__init__()

        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.position_embedding = nn.Embedding(max_position_embeddings, embedding_dim)

        self.rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True,
            bidirectional=bidirectional
        )

        rnn_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.layer_norm = nn.LayerNorm(rnn_output_dim)
        self.fc = nn.Linear(rnn_output_dim, vocab_size)

    def forward(self, x, position_ids, hidden=None):
        """
        x: (batch_size, seq_len)
        position_ids: (batch_size, seq_len)
        """
        tok_emb = self.token_embedding(x)               # (B, T, D)
        pos_emb = self.position_embedding(position_ids) # (B, T, D)
        x = tok_emb + pos_emb

        out, hidden = self.rnn(x, hidden)
        out = self.layer_norm(out)
        out = self.fc(out)

        return out, hidden

Training<br>

In [ ]:
def train(model, train_loader, val_loader, vocab_size, num_epochs=10, lr=1e-4, device='cpu'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            batch = batch['input_ids'].to(device)  # (batch_size, seq_len)

            input_ids = batch[:, :-1]
            target_ids = batch[:, 1:]

            # Create position_ids: simply range from 0 to seq_len-1
            position_ids = torch.arange(input_ids.size(1), device=device).unsqueeze(0).expand_as(input_ids)

            optimizer.zero_grad()
            outputs, _ = model(input_ids, position_ids)
            outputs = outputs.reshape(-1, vocab_size)
            targets = target_ids.reshape(-1)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # --- Validation ---
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch['input_ids'].to(device)

                input_ids = batch[:, :-1]
                target_ids = batch[:, 1:]
                position_ids = torch.arange(input_ids.size(1), device=device).unsqueeze(0).expand_as(input_ids)

                outputs, _ = model(input_ids, position_ids)
                outputs = outputs.reshape(-1, vocab_size)
                targets = target_ids.reshape(-1)

                loss = criterion(outputs, targets)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


# Example usage
if __name__ == "__main__":
    vocab_size = tokenizer.vocab_size
    embedding_dim = 256
    hidden_dim = 512
    num_layers = 2

    model = MusicRNN(
    vocab_size=tokenizer.vocab_size,
    embedding_dim=256,
    hidden_dim=512,
    num_layers=2,
    dropout=0.3,
    bidirectional=False   # try True if you want to experiment with bidirectional context
)

    train(model, train_loader, test_loader, vocab_size)

Epoch 1/10 | Train Loss: 6.4398 | Val Loss: 5.9706
Epoch 2/10 | Train Loss: 5.8206 | Val Loss: 5.7486
Epoch 3/10 | Train Loss: 5.5928 | Val Loss: 5.5155
Epoch 4/10 | Train Loss: 5.2966 | Val Loss: 5.1142
Epoch 5/10 | Train Loss: 4.8946 | Val Loss: 4.7733
Epoch 6/10 | Train Loss: 4.6610 | Val Loss: 4.6088
Epoch 7/10 | Train Loss: 4.5303 | Val Loss: 4.5059
Epoch 8/10 | Train Loss: 4.4355 | Val Loss: 4.4309
Epoch 9/10 | Train Loss: 4.3591 | Val Loss: 4.3625
Epoch 10/10 | Train Loss: 4.2933 | Val Loss: 4.3050


Sampling<br>

In [ ]:

def sample_top_k(model, start_token, max_length, temperature, k, tokenizer, device='cpu'):
    model = model.to(device)
    model.eval()

    generated = [start_token]
    position_ids = [0]  # start with position 0
    input_token = torch.tensor([[start_token]], device=device)
    input_pos = torch.tensor([[0]], device=device)  # position ID

    hidden = None
    current_position = 0

    for _ in range(max_length):
        output, hidden = model(input_token, input_pos, hidden)  # (1, 1, vocab_size)
        logits = output[:, -1, :] / temperature

        top_k_logits, top_k_indices = torch.topk(logits, k)
        top_k_probs = F.softmax(top_k_logits, dim=-1)

        next_token_idx = torch.multinomial(top_k_probs, 1).item()
        next_token = top_k_indices[0, next_token_idx].item()
        generated.append(next_token)

        # --- Update position ---
        token_str = tokenizer.vocab[next_token]
        if token_str.startswith("Position_"):
            try:
                current_position = int(token_str.split("_")[1])
            except:
                current_position = 0
        position_ids.append(current_position)

        # Stop if EOS or PAD token is hit
        if next_token == tokenizer["EOS_None"] or next_token == tokenizer.pad_token_id:
            break

        # Prepare next input
        input_token = torch.tensor([[next_token]], device=device)
        input_pos = torch.tensor([[current_position]], device=device)

    return generated


start_token = tokenizer["BOS_None"]
generated_sequence = sample_top_k(
    model=model,
    start_token=start_token,
    max_length=1024,
    temperature=0.7,
    k=5,
    tokenizer=tokenizer,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)


Generated token sequence:
[1, 410, 371, 205, 48, 102, 143, 207, 36, 103, 142, 208, 48, 104, 139, 210, 43, 104, 143, 212, 43, 101, 140, 217, 43, 101, 139, 219, 43, 103, 139, 410, 198, 48, 104, 141, 199, 55, 103, 140, 203, 55, 105, 142, 206, 55, 104, 143, 209, 55, 105, 143, 211, 55, 103, 148, 215, 50, 103, 139, 220, 50, 102, 142, 435, 38, 101, 141, 193, 43, 102, 140, 196, 43, 104, 140, 199, 55, 106, 140, 203, 43, 102, 143, 206, 43, 101, 139, 208, 48, 101, 139, 212, 45, 100, 138, 216, 48, 100, 138, 219, 43, 102, 143, 422, 43, 101, 139, 193, 48, 101, 140, 196, 52, 104, 141, 198, 43, 102, 141, 203, 55, 105, 143, 207, 55, 103, 142, 208, 52, 103, 141, 214, 48, 101, 140, 219, 57, 106, 140, 220, 48, 101, 139, 410, 198, 48, 101, 139, 202, 45, 102, 139, 207, 36, 102, 141, 209, 45, 101, 142, 211, 36, 101, 140, 214, 45, 100, 138, 410, 199, 57, 109, 140, 45, 100, 140, 410, 199, 57, 106, 139, 202, 48, 104, 143, 206, 45, 102, 143, 207, 57, 104, 139, 212, 50, 102, 139, 217, 45, 101, 140, 410, 198, 50, 

Generation output of midi files<br>

In [ ]:
def generate_midi(tokenizer, generated_sequence, output_filename="rnn.mid"):
    try:
        vocab_size = tokenizer.vocab_size
        valid_sequence = [token for token in generated_sequence if 0 <= token < vocab_size]
        
        print(f"Original sequence length: {len(generated_sequence)}")
        print(f"Valid sequence length: {len(valid_sequence)}")
        
        if len(valid_sequence) < 2:
            print("Sequence too short or no valid tokens found")
            return None
            
        # Fix: Wrap the sequence in another list
        output_score = tokenizer.decode([valid_sequence])
        
        if len(output_score.tracks) == 0:
            print("Generated MIDI has no tracks")
            return None
            
        output_score.dump_midi(output_filename)
        print(f"Successfully generated {output_filename}")
        return output_score
        
    except Exception as e:
        print(f"Error during MIDI generation: {e}")
        print(f"Sequence sample: {generated_sequence[:20]}...")
        return None


# Usage
start_token = tokenizer.special_tokens_ids[1] if len(tokenizer.special_tokens_ids) > 1 else 1
generated_sequence = sample_top_k(model, start_token, max_length=1024, temperature=0.7, k=5)

# Generate MIDI safely
output_score = generate_midi(tokenizer, generated_sequence, "rnn.mid")


Original sequence length: 1025
Valid sequence length: 1025
Error during MIDI generation: File not found file (error:13): rnn.mid
Sequence sample: [1, 410, 371, 205, 32, 102, 143, 206, 44, 101, 142, 209, 49, 103, 139, 214, 44, 101, 140, 218]...
